In [1]:
# %matplotlib inline

import matplotlib
matplotlib.use('pgf')
pgf_with_rc_fonts = {
    "font.family": "serif",
    "font.serif": [],                   # use latex default serif font
    "font.sans-serif": ["DejaVu Sans"], # use a specific sans-serif font
    "pgf.rcfonts" : False
}
matplotlib.rcParams.update(pgf_with_rc_fonts)

from matplotlib import rc
import matplotlib.pyplot as plt

import glob
import os
import scipy.misc, scipy.stats
import numpy as np
import json

from skimage.draw import line_aa

import sys
sys.path.append("/home/joshua/Documents/Uni/Year4/dissertation/catkin_ws/src/linefollow_gazebo/scripts")

import helper as helper
import CameraModel as CameraModel

plt.style.use('ggplot')
from matplotlib import rc

rc('text', usetex=True)
matplotlib.rcParams['axes.facecolor'] = 'white'
matplotlib.rcParams['axes.edgecolor'] = '0.4'
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['grid.color'] = "0.9"

plt.tight_layout(.5)



In [2]:

def get_predicted_error_radius(camera_height,
                               ground_distance, 
                               pixel_area,
                               sigma_pos=0.03/2.0,
                               sigma_orient_deg=0.5/2,
                               alg_error=0.0,
                               verbose=False,
                               orientation_percent=0.9,
                               pos_percent=0.9,
                               alg_percent=0.9,
                               use_err_xy=True,
                               use_err_z=True,
                               use_err_orient=True,
                               use_err_pixel=True
                              ):
    stddevs_orientation = scipy.stats.norm.ppf(orientation_percent)
    stddevs_position = scipy.stats.norm.ppf(pos_percent)
    
    stddevs_alg = scipy.stats.norm.ppf(alg_percent)
    
    # 95% of positional ground error will have camera position error less than +-0.03
    # given are 1 stddev, want to use two
    xy_error = 2 * stddevs_position * sigma_pos if use_err_xy else 0
    z_error = (ground_distance/camera_height)*stddevs_position*sigma_pos if use_err_z else 0
        
    if use_err_orient:
        t = stddevs_orientation*np.deg2rad(sigma_orient_deg)
        alpha = np.arctan2(ground_distance, camera_height)
        orientation_error = np.tan(alpha + t)*camera_height - ground_distance
    else:
        orientation_error = 0
        
    pixel_error = np.sqrt(2*pixel_area) if use_err_pixel else 0
    
    if verbose:
        print("Position radius: {0}, orientation radius: {2}, pixel radius: {1}".format(position_error, pixel_error, orientation_error))
    total = xy_error + z_error + orientation_error + (alg_error+1)/2.0 * stddevs_alg * pixel_error
    return total, xy_error, z_error, orientation_error, alg_error*pixel_error
    
    

In [3]:

    
   

def do_eval(real_position, 
            real_pitch_deg,
            real_yaw_deg,
            sigma_pos=0.03/2.0,
            sigma_orient_deg=0.5/2,
            alg_error=0.0,
            camera_res=(500,500),
            camera_fov=(60.0, 60.0),
            n_samples_per_camera=10,
            n_camera_instances=1,
            orientation_percent=0.90,
            pos_percent=0.9,
            alg_percent=0.9,
            use_err_xy=True,
            use_err_z=True,
            use_err_orient=True,
            use_err_pixel=True,
            verbose=False):
    
    real_camera = CameraModel.Camera(position=real_position, orientation_pitch_deg=real_pitch_deg, 
                                     orientation_yaw_deg=real_yaw_deg, verbose=False)
    real_camera.set_resolution(*camera_res)
    real_camera.set_fov(*camera_fov)

    results = {
        'true_vehicle_center':[], # placed using real_camera
        'calculated_vehicle_center':[], # project world -> biased camera -> round to pixel -> world
        'vehicle_center_prediction_error':[], # difference between above two
        'predicted_error_radius':[],
        'camera_biases':[], # enter tuples of (xyz, rp) bias
        'xy_errors': [],
        'z_errors': [],
        'orientation_errors': [],
        'pixel_alg_errors': []
    }
    for _ in range(n_camera_instances):
        offset_xyz = np.random.normal(loc=0.0, scale=sigma_pos, size=3)
        offset_rp = np.random.normal(loc=0.0, scale=sigma_orient_deg, size=2)
        biased_camera = CameraModel.Camera(position=real_position+offset_xyz, 
                                           orientation_pitch_deg=real_pitch_deg+offset_rp[0], 
                                           orientation_yaw_deg=real_yaw_deg+offset_rp[1],
                                           verbose=False
                                          )
        biased_camera.set_resolution(*camera_res)
        biased_camera.set_fov(*camera_fov)
        

        
        true_centers, calculated_centers, prediction_error, predicted_radius = [],[],[],[]
        xy_errors, z_errors, orientation_errors, pixel_alg_errors = [], [], [], []
        for i in range(n_samples_per_camera):
            
            # pick a uniformly distributed pixel to project through
            x,y = np.random.uniform(0, camera_res)
            # project through real camera to world coordinate to get something likely to be in FoV
            pt = real_camera.pixel_to_plane(x, y)
            true_centers.append(pt)
            
            # project world coordinate through biased camera
            pixel = biased_camera.world_to_pixel(*pt)
            # round to nearest pixel
            pixel = np.round(pixel)
            
            # TODO implement algorithm error
            
            pixel_translation_distance = np.random.normal(loc=0.0, scale=alg_error/2.0)
            pixel_translation_direction = np.random.uniform(0.0, 2*np.pi)
            dx = pixel_translation_distance * np.cos(pixel_translation_direction)
            dy = pixel_translation_distance * np.sin(pixel_translation_direction)
            
            pixel += np.array([dx, dy])
            
            
#             print("True Pixel: ({0}, {1}), using pixel: {2}".format(x, y, pixel))
            pixel_area = real_camera.plane_area_of_pixel(*pixel) # get area in true camera (will be wrong pixel)
            # project back onto ground to get estimate as if it were the true camera
            world_estimate = real_camera.pixel_to_plane(*pixel)
            calculated_centers.append(world_estimate)
            
            if pt is None or world_estimate is None:
                continue
            # error
            error = np.linalg.norm(world_estimate - pt)
#             print("=> World plane error: \t\t{0}".format(error))
            prediction_error.append(error)
            
            # predicted error radius
            radius, xy_err, z_err, orientation_err, pixel_err  = get_predicted_error_radius(real_position[2],
                               ground_distance=np.linalg.norm(world_estimate[:2]), 
                               pixel_area=pixel_area,
                               sigma_pos=sigma_pos,
                               sigma_orient_deg=sigma_orient_deg,
                               alg_error=alg_error,
                               verbose=verbose,             
                               orientation_percent=orientation_percent,
                               pos_percent=pos_percent, 
                               use_err_xy=use_err_xy,
                               use_err_z=use_err_z,
                               use_err_orient=use_err_orient,
                               use_err_pixel=use_err_pixel
                              )
            
            predicted_radius.append(radius)
            xy_errors.append(xy_err)
            z_errors.append(z_err)
            orientation_errors.append(orientation_err)
            pixel_alg_errors.append(pixel_err)
            
#             print("=> Predicted error radius:  \t\t{0}".format(radius))
            
            
            
            if verbose and radius < error:
                print("Offset XYZ: {0}, RP: {1}".format(offset_xyz, offset_rp))
                print("True pixel & ground area: {0}, {1}".format((x,y), real_camera.plane_area_of_pixel(*np.round([x,y]))))
                print("True world pos: {0}".format(pt))
                print("Biased pixel: {0}".format(pixel))
                print("Pixel Ground area in true camera: {0}".format(pixel_area))
                print("World pos estimate: {0}".format(world_estimate))
                print("Error: {0}".format(error))
                print("Predicted Error Radius: {0}".format(radius))
                print("-----------------")
        results['true_vehicle_center'].append(np.array(true_centers))
        results['calculated_vehicle_center'].append(np.array(calculated_centers))
        results['vehicle_center_prediction_error'].append(np.array(prediction_error))
        results['predicted_error_radius'].append(np.array(predicted_radius))
        results['camera_biases'].append(np.array([offset_xyz, offset_rp]))
        results['xy_errors'].append(xy_errors)
        results['z_errors'].append(z_errors)
        results['orientation_errors'].append(orientation_errors)
        results['pixel_alg_errors'].append(pixel_alg_errors)

    results['true_vehicle_center'] = np.array(results['true_vehicle_center'])
    results['calculated_vehicle_center'] = np.array(results['calculated_vehicle_center'])
    results['vehicle_center_prediction_error'] = np.array(results['vehicle_center_prediction_error'])
    results['predicted_error_radius'] = np.array(results['predicted_error_radius'])
    results['xy_errors'] = np.array(results['xy_errors'])
    results['z_errors'] = np.array(results['z_errors']) 
    results['orientation_errors'] = np.array(results['orientation_errors'])
    results['pixel_alg_errors'] = np.array(results['pixel_alg_errors'])
    results['camera_biases'] = np.array(results['camera_biases'])

    return results




In [4]:
def evaluate_error_model(camera_height_range=(4.0, 10.0), 
                         camera_pitch_range=(32.0, 90.0), 
                         camera_pos_samples=100, 
                         camera_instances_per=50, 
                         samples_per_camera=50, 
                         sigma_pos=0.03/2, 
                         sigma_orient_deg=0.5/2, 
                         alg_error=1.0, 
                         use_err_xy=True,
                         use_err_z=True,
                         use_err_orient=True,
                         use_err_pixel=True,      
                         orientation_percent=0.9,
                         pos_percent=0.9, 
                         alg_percent=0.9
                        ):
    all_results = []
    
    for _ in range(camera_pos_samples):
        height, pitch = np.random.uniform(*np.array([camera_height_range, camera_pitch_range]).T)
        all_results.append(
            do_eval(np.array([0.0, 0.0, height]), 
                pitch,
                0.0,
                sigma_pos=sigma_pos,
                sigma_orient_deg=sigma_orient_deg,
                alg_error=alg_error,
                camera_res=(500,500),
                camera_fov=(60.0, 60.0),
                n_samples_per_camera=samples_per_camera,
                n_camera_instances=camera_instances_per,
                orientation_percent=orientation_percent,
                pos_percent=pos_percent, 
                alg_percent=alg_percent,
                use_err_xy=use_err_xy,
                use_err_z=use_err_z,
                use_err_orient=use_err_orient,
                use_err_pixel=use_err_pixel
            )
        )
    return all_results
    
    

In [5]:
outlier_cutoff = 3.0

### 1 show without pixel error, radii are gaussian...ish

In [6]:
no_pixel = evaluate_error_model(camera_pos_samples=100, camera_instances_per=50, samples_per_camera=25,
                                use_err_pixel=False)

In [7]:
plt.clf()
plt.figure(figsize=(6,3))
all_radii_no_pixel = np.array([res['predicted_error_radius'] for res in no_pixel]).ravel()
outliers = all_radii_no_pixel > outlier_cutoff
num_outliers, total = np.count_nonzero(outliers), all_radii_no_pixel.shape[0]
non_outliers = all_radii_no_pixel[~outliers]
plt.hist(non_outliers, bins=200)
print("Number of outliers (>{3}) discarded: {0}/{1} ... {2}".format(num_outliers, total, num_outliers/float(total), outlier_cutoff))
print("Mean: {0}, stddev: {1}".format(np.mean(non_outliers), np.var(non_outliers)**2 ))
plt.tight_layout()
plt.savefig("figs/no-pixel-error.pdf",bbox_inches='tight')
plt.clf()

Number of outliers (>3.0) discarded: 520/125000 ... 0.00416
Mean: 0.174188602986, stddev: 0.00264567702634


### 1.5 show with pixel error, radii are less gaussian (linear error as a function of ) hmmm... ignore
This is actually more gaussian...

In [8]:
with_pixel_err = evaluate_error_model(camera_pos_samples=100, camera_instances_per=50, samples_per_camera=25,
                                use_err_pixel=True)

In [9]:
plt.clf()
plt.figure(figsize=(6,3))

all_radii_with_pixel = np.array([res['predicted_error_radius'] for res in with_pixel_err]).ravel()
outliers = all_radii_with_pixel > outlier_cutoff
num_outliers, total = np.count_nonzero(outliers), all_radii_with_pixel.shape[0]
non_outliers = all_radii_with_pixel[~outliers]
plt.hist(non_outliers, bins=200)
print("Number of outliers (>{3}) discarded: {0}/{1} ... {2}".format(num_outliers, total, num_outliers/float(total), outlier_cutoff))
print("Mean: {0}".format(np.mean(non_outliers)))
print("stddev: {0}".format(np.var(non_outliers)**2 ))
print("Median: {0}".format(np.median(non_outliers)))

plt.ylabel("Frequency")
plt.xlabel("Error Radius (meters)")
plt.tight_layout()
plt.savefig("figs/error_radius_distribution.pdf",bbox_inches='tight')
plt.savefig("figs/error_radius_distribution.pgf",bbox_inches='tight')

Number of outliers (>3.0) discarded: 1319/125000 ... 0.010552
Mean: 0.226132189004
stddev: 0.00595110782175
Median: 0.142596650593


### 2. Show that most of the true errors are less than the predicted ones

In [10]:
sampled_results = evaluate_error_model(camera_pos_samples=50, camera_instances_per=50, samples_per_camera=50)

In [11]:
all_errors = np.array([res['vehicle_center_prediction_error'] for res in sampled_results]).ravel()
all_radii = np.array([res['predicted_error_radius'] for res in sampled_results]).ravel()
outlier_cutoff_2 = 99999
outliers = all_radii > outlier_cutoff_2 # remove all the ones with too high of a predicted error radius

# remove outliers
errors = all_errors[~outliers]
radii = all_radii[~outliers]

# count correctly within radius
errs_within_radii = errors < radii
n_within, n_total = np.count_nonzero(errs_within_radii), all_radii.shape[0]

print("Errors within predicted error radius: {0}/{1} ... {2}".format(n_within, n_total, float(n_within)/n_total))
print("Errors not within error radius: {0}/{1} ... {2}".format(n_total-n_within, n_total, float(n_total-n_within)/n_total))



Errors within predicted error radius: 124125/125000 ... 0.993
Errors not within error radius: 875/125000 ... 0.007


In [12]:
print("outliers removed (>{0}): {1} ... {2}".format(outlier_cutoff, np.count_nonzero(outliers), np.count_nonzero(outliers)/float(outliers.shape[0])))

outliers removed (>3.0): 0 ... 0.0


In [13]:
plt.clf()
plt.figure(figsize=(6,3))

# examine distribution of differences
diffs = radii - errors
inside_diffs = diffs[diffs > 0]
outside_diffs = diffs[diffs < 0]
print("Mean positive difference: {0}".format(np.mean(diffs[diffs > 0])))
_ = plt.hist(inside_diffs, bins=300)
_ = plt.hist(outside_diffs, bins=300)
plt.xlim(-0.4, 1.0)
# plt.ylim(-1, 1000)
_ = plt.xlabel("Difference (Error Radius - True Error)")
_ = plt.ylabel("Occurrences")

plt.tight_layout()
plt.savefig('figs/diff_radius_true_error.pdf',bbox_inches='tight')
plt.savefig('figs/diff_radius_true_error.pgf',bbox_inches='tight')

Mean positive difference: 0.165606792681


In [14]:
plt.clf()
plt.figure(figsize=(6,3))

percent_diff = (diffs / radii)
negative_percents = percent_diff[percent_diff < 0]
positive_percents = percent_diff[percent_diff >= 0]
plt.xlim(-1, 1)
_ = plt.xlabel("Predicted Error Radius - True Error as %")
_ = plt.ylabel("Occurrences")
_ = plt.hist(positive_percents, bins=50)
_ = plt.hist(negative_percents, bins=50)

plt.tight_layout()
plt.savefig('figs/proportion_radius_true_error.pdf',bbox_inches='tight')
plt.savefig('figs/proportion_radius_true_error.pgf',bbox_inches='tight')

### 3. How components of the error vary as the pitch angle changes
1. instantiate camera at official angle 90 pitch -> 40 pitch, at 20 degree increments
2. randomly sample and collect components of the error
3. Show how the components grow on average as pitch angle changes (3 side by side bars for easy visual comparison)


In [15]:
vary_pitch_results = []
for pitch in np.arange(31.0, 92.0, 10.0):
    vary_pitch_results.append(do_eval(np.array([0.0, 0.0, 6.0]), 
                      pitch,
                      0.0,              
                      sigma_pos=0.03/2.0,
                      sigma_orient_deg=0.05/2,
                      n_samples_per_camera=50,
                      n_camera_instances=50,
                      orientation_percent=0.9,
                      pos_percent=0.9,
                      alg_error=1.0,
                    )
    )
    
    # just spit out the error < radius stats
    radii = vary_pitch_results[-1]['predicted_error_radius'].ravel()
    errors = vary_pitch_results[-1]['vehicle_center_prediction_error'].ravel()
    within = errors < radii
    biases = vary_pitch_results[-1]['camera_biases']
    #print("Camera shifts and skews: {0}, {1}".format(biases[:, 0], biases[:, 1]))
    print("Proportion of errors within radius at pitch angle {0}: {1}".format(pitch, np.count_nonzero(within)/float(radii.shape[0])))
    print("Mean radius: {0}".format(np.mean(radii)))
    print("Median radius: {0}\n\n".format(np.median(radii)))
    
#     _ = plt.hist((radii-errors), bins=50)
#     plt.figure()
    

Proportion of errors within radius at pitch angle 31.0: 0.9912
Mean radius: 0.654048418959
Median radius: 0.155027343585


Proportion of errors within radius at pitch angle 41.0: 0.9996
Mean radius: 0.153089851946
Median radius: 0.116631356159


Proportion of errors within radius at pitch angle 51.0: 0.9996
Mean radius: 0.111030120009
Median radius: 0.0980730119737


Proportion of errors within radius at pitch angle 61.0: 1.0
Mean radius: 0.0925422000212
Median radius: 0.0875185567687


Proportion of errors within radius at pitch angle 71.0: 1.0
Mean radius: 0.0824987618637
Median radius: 0.0798693529603


Proportion of errors within radius at pitch angle 81.0: 1.0
Mean radius: 0.0775333671075
Median radius: 0.0763677296715


Proportion of errors within radius at pitch angle 91.0: 1.0
Mean radius: 0.0761440924118
Median radius: 0.0764216658718




In [16]:
outlier_cutoff = 5.0
# outlier_cutoff = 99999999

plt.clf()

vary_pitch_stats_summary = {}
for index, result in enumerate(vary_pitch_results):
    angle = 31 + 10*index
    vary_pitch_stats_summary[angle] = {}
    
    total_error_radii = result['predicted_error_radius'].ravel()
    outliers = total_error_radii > outlier_cutoff
    total_error_radii = total_error_radii[~outliers]
    n_outliers, n_total = np.count_nonzero(outliers), total_error_radii.shape[0]
    print("Number of outliers removed (> {0}): {1}/{2} ... {3}".format(outlier_cutoff, n_outliers, n_total, n_outliers/float(n_total)))
    mean, median, stddev = np.mean(total_error_radii), np.median(total_error_radii), np.var(total_error_radii)**2
    vary_pitch_stats_summary[angle]['total_error'] = {
        'mean': mean,
        "media": median,
        '99th percentile': np.percentile(total_error_radii, 99.0),
        "stddev": stddev
    }
    
    xy_errs = result['xy_errors'].ravel() # constant
    xy_errs = xy_errs[~outliers]
    mean_xy, med_xy, stddev_xy = np.mean(xy_errs), np.median(xy_errs), np.var(xy_errs)**2
    percentile_xy = np.percentile(xy_errs, 99.0)
    vary_pitch_stats_summary[angle]['XY Error Radius'] = {
        'mean': mean_xy,
        'median': med_xy,
        '99th percentile': percentile_xy,
        'stddev': stddev_xy
    }
    
    z_errs = result['z_errors'].ravel()
    z_errs = z_errs[~outliers]
    mean_z, med_z, stddev_z = np.mean(z_errs), np.median(z_errs), np.var(z_errs)**2
    percentile_z = np.percentile(z_errs, 99.0)
    vary_pitch_stats_summary[angle]['Z Error Radius']= {
        'mean': mean_z,
        'median': med_z,
        '99th percentile': percentile_z,
        'stddev': stddev_z
    }

    orient_errs = result['orientation_errors'].ravel()
    orient_errs = orient_errs[~outliers]
    mean_or, med_or, stddev_or = np.mean(orient_errs), np.median(orient_errs), np.var(orient_errs)**2
    percentile_or = np.percentile(orient_errs, 99.0)
    vary_pitch_stats_summary[angle]['Orientation Error Radius']= {
        'mean': mean_or,
        'median': med_or,
        '99th percentile': percentile_or,
        'stddev': stddev_or
    }

    pixel_alg_errs = result['pixel_alg_errors'].ravel()
    pixel_alg_errs = pixel_alg_errs[~outliers]
    mean_p, med_p, stddev_p = np.mean(pixel_alg_errs), np.median(pixel_alg_errs), np.var(pixel_alg_errs)**2
    percentile_p = np.percentile(pixel_alg_errs, 99.0)
    vary_pitch_stats_summary[angle]['Pixel and Algorithmic Error Radius']= {
        'mean': mean_p,
        'median': med_p,
        '99th percentile': percentile_p,
        'stddev': stddev_p
    }
    
    mean_total = mean_xy + mean_z + mean_p + mean_or
    xy_mean_norm, z_mean_norm, or_mean_norm, p_mean_norm = mean_xy/mean_total, mean_z/mean_total, mean_or/mean_total, mean_p/mean_total
    
    med_total = med_xy + med_z + med_p + med_or
    xy_med_norm, z_med_norm, or_med_norm, p_med_norm = med_xy/med_total, med_z/med_total,med_or/med_total, med_p/med_total
    

    percentile_total = med_xy + med_z + med_p + med_or
    xy_med_norm, z_med_norm, or_med_norm, p_med_norm = med_xy/med_total, med_z/med_total,med_or/med_total, med_p/med_total
    
    percentile_total = percentile_xy + percentile_z + percentile_p + percentile_or
    xy_perc_norm, z_perc_norm, or_perc_norm, p_perc_norm = percentile_xy/percentile_total, percentile_z/percentile_total, percentile_or/percentile_total, percentile_p/percentile_total
    
    vary_pitch_stats_summary[angle]["normalized_summary"] = {
        "means": {
            "xy error" : xy_mean_norm,
            "z error": z_mean_norm,
            "orientation error": or_mean_norm,
            "pixel and algorithm error": p_mean_norm
        },
        "medians": {
            "xy error" : xy_med_norm,
            "z error": z_med_norm,
            "orientation error": or_med_norm,
            "pixel and algorithm error": p_med_norm
        },
        "99th percentile": {
            "xy error" : xy_perc_norm,
            "z error": z_perc_norm,
            "orientation error": or_perc_norm,
            "pixel and algorithm error": p_perc_norm
        }
    }
    # TODO stick these in a summary dict
    # TODO make them into a figure of relative important as a function of pitch angle as a sanity check

Number of outliers removed (> 5.0): 68/2432 ... 0.0279605263158
Number of outliers removed (> 5.0): 0/2500 ... 0.0
Number of outliers removed (> 5.0): 0/2500 ... 0.0
Number of outliers removed (> 5.0): 0/2500 ... 0.0
Number of outliers removed (> 5.0): 0/2500 ... 0.0
Number of outliers removed (> 5.0): 0/2500 ... 0.0
Number of outliers removed (> 5.0): 0/2500 ... 0.0


In [17]:
print(json.dumps(vary_pitch_stats_summary, sort_keys=True, indent=4))

{
    "31": {
        "Orientation Error Radius": {
            "99th percentile": 1.4426433948016613, 
            "mean": 0.092042718840877266, 
            "median": 0.01346992245908929, 
            "stddev": 0.0037718827347664962
        }, 
        "Pixel and Algorithmic Error Radius": {
            "99th percentile": 1.4578159956505452, 
            "mean": 0.14620372168203433, 
            "median": 0.051544796959808542, 
            "stddev": 0.0045432050671552491
        }, 
        "XY Error Radius": {
            "99th percentile": 0.038446546966338008, 
            "mean": 0.038446546966338001, 
            "median": 0.038446546966338008, 
            "stddev": 2.3182538441796384e-69
        }, 
        "Z Error Radius": {
            "99th percentile": 0.39583384922755094, 
            "mean": 0.062618533621246183, 
            "median": 0.03335596059121769, 
            "stddev": 3.3319588248813447e-05
        }, 
        "normalized_summary": {
            "99th percent

In [18]:
def sidebyside_bar(series, xlabels, series_labels, ylabel, xlabel, error_bars=None, label_suffix="", label_multiplier=1, label_decimals=2, figsize=(8,6), width=0.27, label_height_offset_mult=1.0):
    if error_bars is None:
        error_bars = np.zeros_like(series)
        
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    
    n = series.shape[1]
    ind = (np.arange(n) - 0.5) * (width/0.27)
    
    width = width * 3 / len(series_labels)
    
    rects = []
    for index, data in enumerate(series):
        yerr = error_bars[index]
        if yerr is None:
            yerr = np.zeros_like(data)
        rects.append(ax.bar(ind + width*index, data, width, yerr=yerr))
        
    ax.set_ylabel(ylabel)
    ax.set_xticks(ind + 0.5 * (len(series_labels) -1) *width)
    ax.set_xticklabels( xlabels )
    ax.legend( rects, series_labels)
    ax.set_xlabel(xlabel)
    
    def autolabel(rects):
        for rect in rects:
            h = rect.get_height()
            ax.text(rect.get_x()+rect.get_width()/2., h+0.005*label_height_offset_mult, ('%.'+str(label_decimals)+'f')%float(h*label_multiplier) + label_suffix,
                    ha='center', va='bottom')
    
    for r in rects:
        autolabel(r)
    
    
    

In [19]:
plt.clf()

plt.figure(figsize=(6,3))

# plot absolute errors -- this is the most informative! Using means tells the best story, but tell reader to be aware of massive skews

angles = [90, 70, 50, 40, 30]
xy = np.array([vary_pitch_stats_summary[angle + 1]["XY Error Radius"]['mean'] for angle in angles])
z =  np.array([vary_pitch_stats_summary[angle + 1]["Z Error Radius"]["mean"] for angle in angles])
orientation = np.array([vary_pitch_stats_summary[angle + 1]["Orientation Error Radius"]["mean"] for angle in angles])
pixel = np.array([vary_pitch_stats_summary[angle + 1]["Pixel and Algorithmic Error Radius"]["mean"] for angle in angles])
legend = ["Mean XY Radius", "Mean Z Radius", "Mean Orientation Radius", "Mean Pixel Radius"]

xticks = [u"${}^{{\circ}}$".format(angle) for angle in angles]
width = 0.4
ind = np.arange(0, len(angles))
bars = []
bars.append(plt.bar(ind, xy, width))
bars.append(plt.bar(ind, z, width, bottom=xy))
bars.append(plt.bar(ind, orientation, width, bottom=xy+z))
bars.append(plt.bar(ind, pixel, width, bottom=xy+z+orientation))

plt.ylabel("Mean Predicted Error Radius (meters)")
plt.xlabel("Pitch Angle")
plt.xticks(ind, xticks)
plt.legend(bars, legend)

# sidebyside_bar(np.array([xy, z, orientation, pixel]), xticks, legend, "Mean Predicted Error Radius (cm)", "Pitch Angle", label_decimals=1, label_multiplier=100, figsize=(8,4), label_height_offset_mult=0.25)

plt.tight_layout()

plt.savefig('figs/mean_error_pitch_variation.pdf',bbox_inches='tight')
plt.savefig('figs/mean_error_pitch_variation.pgf',bbox_inches='tight')
plt.show()

In [20]:
# plot absolute errors -- this is the quite informative!
angles = [90, 70, 50, 40, 30]
xy = np.array([vary_pitch_stats_summary[angle + 1]["XY Error Radius"]['median'] for angle in angles])
z =  np.array([vary_pitch_stats_summary[angle + 1]["Z Error Radius"]["median"] for angle in angles])
orientation = np.array([vary_pitch_stats_summary[angle + 1]["Orientation Error Radius"]["median"] for angle in angles])
pixel = np.array([vary_pitch_stats_summary[angle + 1]["Pixel and Algorithmic Error Radius"]["median"] for angle in angles])
legend = ["XY Error (cm)", "Z Error (cm)", "Orientation Error (cm)", "Pixel Error (cm)"]

xticks = [u"${}^{{\circ}}$".format(angle) for angle in angles]
sidebyside_bar(np.array([xy, z, orientation, pixel]), xticks, legend, "Median Predicted Error Radius (cm)", "Pitch Angle", label_decimals=1, label_multiplier=100, label_height_offset_mult=0.1)

plt.tight_layout()

plt.savefig('figs/median_error_pitch_variation.pdf',bbox_inches='tight')
plt.savefig('figs/median_error_pitch_variation.pgf',bbox_inches='tight')

In [21]:
# plot absolute errors -- this is the most informative!
angles = [90, 70, 50, 40, 30]
xy = np.array([vary_pitch_stats_summary[angle + 1]["XY Error Radius"]['99th percentile'] for angle in angles])
z =  np.array([vary_pitch_stats_summary[angle + 1]["Z Error Radius"]["99th percentile"] for angle in angles])
orientation = np.array([vary_pitch_stats_summary[angle + 1]["Orientation Error Radius"]["99th percentile"] for angle in angles])
pixel = np.array([vary_pitch_stats_summary[angle + 1]["Pixel and Algorithmic Error Radius"]["99th percentile"] for angle in angles])
legend = ["XY Error", "Z Error", "Orientation Error", "Pixel Error"]

xticks = [u"${}^{{\circ}}$".format(angle) for angle in angles]
sidebyside_bar(np.array([xy, z, orientation, pixel]), angles, legend, u"99$^{{th}}$ Percentile Predicted Error Radius (cm)", "Pitch Angle", label_decimals=1, label_multiplier=100, figsize=(12,6), label_height_offset_mult=0.25)

plt.tight_layout()

plt.savefig('figs/99th_percentile_error_pitch_variation.pdf',bbox_inches='tight')
plt.savefig('figs/99th_percentile_error_pitch_variation.pgf',bbox_inches='tight')

In [22]:
# collect normalized summaries
# means first

angles = [90, 70, 50, 40, 30]
xy_means = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["means"]["xy error"] for angle in angles])
z_means = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["means"]["z error"] for angle in angles])
orientation_means =  np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["means"]["orientation error"] for angle in angles])
pixel_means = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["means"]["pixel and algorithm error"] for angle in angles])
legend = ["XY Error", "Z Error", "Orientation Error", "Pixel Error"]

xticks = [u"${}^{{\circ}}$".format(angle) for angle in angles]
sidebyside_bar(np.array([xy_means, z_means, orientation_means, pixel_means]), xticks, legend, "Proportion of Predicted Error Radius", "Pitch Angle", label_suffix="%", label_decimals=0, label_multiplier=100, figsize=(12,6))

plt.tight_layout()
plt.savefig('figs/normalized_mean_error_pitch_variation.pdf',bbox_inches='tight')
plt.savefig('figs/normalized_mean_error_pitch_variation.pgf',bbox_inches='tight')

In [23]:
# less useful!
# angles = [90, 80, 70, 60, 50, 40, 30]
xy_medians = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["medians"]["xy error"] for angle in angles])
z_medians = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["medians"]["z error"] for angle in angles])
orientation_medians =  np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["medians"]["orientation error"] for angle in angles])
pixel_medians = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["medians"]["pixel and algorithm error"] for angle in angles])
legend = ["XY Error", "Z Error", "Orientation Error", "Pixel Error"]

sidebyside_bar(np.array([xy_medians, z_medians, orientation_medians, pixel_medians]), xticks, legend, "Median Proportion of Predicted Error Radius", "Pitch Angle", label_suffix="%", label_multiplier=100, label_decimals=0, figsize=(12,6))

plt.tight_layout()
plt.savefig('figs/normalized_median_error_pitch_variation.pdf',bbox_inches='tight')
plt.savefig('figs/normalized_median_error_pitch_variation.pgf',bbox_inches='tight')

In [24]:
# normalized 99th percentile important
xy_99 = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["99th percentile"]["xy error"] for angle in angles])
z_99 = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["99th percentile"]["z error"] for angle in angles])
orientation_99 =  np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["99th percentile"]["orientation error"] for angle in angles])
pixel_99 = np.array([vary_pitch_stats_summary[angle + 1]["normalized_summary"]["99th percentile"]["pixel and algorithm error"] for angle in angles])
legend = ["XY Error", "Z Error", "Orientation Error", "Pixel Error"]

sidebyside_bar(np.array([xy_99, z_99, orientation_99, pixel_99]), xticks, legend, u"99$^{{th}}$ Percentile Error Proportions", "Pitch Angle", label_suffix="%", label_multiplier=100, label_decimals=0, figsize=(12,6))

plt.tight_layout()
plt.savefig('figs/normalized_99th_error_pitch_variation.pdf',bbox_inches='tight')
plt.savefig('figs/normalized_99th_error_pitch_variation.pgf',bbox_inches='tight')

In [25]:
# show absolute errors a totals... also less useful
# angles = [90, 80, 70, 60, 50, 40, 30]


mean_error = np.array([vary_pitch_stats_summary[angle + 1]["total_error"]["mean"] for angle in angles])
median_error = np.array([vary_pitch_stats_summary[angle + 1]["total_error"]["media"] for angle in angles])
stddev_error = np.array([vary_pitch_stats_summary[angle + 1]["total_error"]["stddev"] for angle in angles])
percentile_99 = np.array([vary_pitch_stats_summary[angle + 1]["total_error"]['99th percentile'] for angle in angles])
three_stddev = mean_error + 3*stddev_error
legend=["Mean Error Radius (cm)", "Median Error Radius (cm)", "99th Percentile", "Mean + 3 stddev"]
print(stddev_error)
"""

sidebyside_bar(np.array([mean_error, median_error, percentile_99, three_stddev]), angles, legend, "Total Radius", "Pitch Angle", label_decimals=1, width=0.4, label_multiplier=100)
# error bars are nothing to write home about except that divergence at the top end - discuss in TABLE

"""

[  1.91543855e-10   2.23365635e-08   2.22067899e-06   7.01101385e-05
   1.80141603e-01]


'\n\nsidebyside_bar(np.array([mean_error, median_error, percentile_99, three_stddev]), angles, legend, "Total Radius", "Pitch Angle", label_decimals=1, width=0.4, label_multiplier=100)\n# error bars are nothing to write home about except that divergence at the top end - discuss in TABLE\n\n'

### 4. Show impact of algorithmic error

In [26]:
outlier_cutoff = 4.0
# want to see how mean/median radius changes, as well as bound accuracy
# as algorithmic error increases
alg_means = []
alg_medians = []
alg_99th = []
alg_outliers = []
alg_within_radius = []
alg_total = []
alg_stddevs = []
for alg_error in np.arange(0.0, 25.0, 2.5):
    with_alg = evaluate_error_model(camera_pos_samples=30, camera_instances_per=30, samples_per_camera=50, alg_error=alg_error)
    with_alg_errors = np.array([res['vehicle_center_prediction_error'] for res in with_alg]).ravel()
    with_alg_radii = np.array([res['predicted_error_radius'] for res in with_alg]).ravel()
    outliers = with_alg_radii > outlier_cutoff # remove all the ones with too high of a predicted error radius
    n_outliers = np.count_nonzero(outliers)
    alg_outliers.append(n_outliers)
    print("Number of outliers removed: {0}/{1}".format(n_outliers, outliers.shape[0]))
    
    # remove outliers
    errors = with_alg_errors[~outliers]
    radii = with_alg_radii[~outliers]
    
    # count correctly within radius
    errs_within_radii = errors < radii
    n_within, n_total = np.count_nonzero(errs_within_radii), errors.shape[0]
    alg_within_radius.append(n_within)
    alg_total.append(n_total)
    print("Errors within predicted error radius: {0}/{1} ... {2}".format(n_within, n_total, float(n_within)/n_total))
    print("Errors not within error radius: {0}/{1} ... {2}".format(n_total-n_within, n_total, float(n_total-n_within)/n_total))
    
    mean, median, percentile, stddev = np.mean(radii), np.median(radii), np.percentile(radii, 99.0), np.var(radii)**2
    print("Mean radius: {0}".format(mean))
    print("Median radius: {0}".format(median))
    print("99th Percentile: {0}".format(percentile))
    print("Stddev: {0}".format(stddev))
    alg_means.append(mean)
    alg_medians.append(median)
    alg_99th.append(percentile)
    alg_stddevs.append(stddev)

Number of outliers removed: 61/45000
Errors within predicted error radius: 44203/44939 ... 0.983622243486
Errors not within error radius: 736/44939 ... 0.0163777565144
Mean radius: 0.183208251906
Median radius: 0.12335743449
99th Percentile: 1.22280848756
Stddev: 0.00321304173468
Number of outliers removed: 114/45000
Errors within predicted error radius: 44765/44886 ... 0.997304281959
Errors not within error radius: 121/44886 ... 0.00269571804126
Mean radius: 0.228954169583
Median radius: 0.17281256248
99th Percentile: 1.23254318759
Stddev: 0.0021998259207
Number of outliers removed: 177/45000
Errors within predicted error radius: 44625/44823 ... 0.995582624992
Errors not within error radius: 198/44823 ... 0.00441737500837
Mean radius: 0.309179995136
Median radius: 0.202298425172
99th Percentile: 2.10895949534
Stddev: 0.0146834932184
Number of outliers removed: 184/45000
Errors within predicted error radius: 44567/44816 ... 0.99444394859
Errors not within error radius: 249/44816 ... 0.

In [27]:
# Here, plot mean radius versus algo error
# as well as bound accuracy versus algorithm error 
# plot lines on the same plot

plt.clf()
plt.figure(figsize=(6,3))
xaxis = np.arange(0.0, 25.0, 2.5)
xticks = ["{0:.2f}".format(p) for p in np.arange(0.0, 25.0, 2.5)]
# plot lines and errors for mean
plt.errorbar(xaxis, alg_means, yerr=3*np.array(alg_stddevs), fmt='-o')

# calculate fraction within error bound
within = np.array(alg_within_radius)
total = np.array(alg_total)
frac = within/total.astype(np.float64)
plt.plot(xaxis, frac, '-x')
    
leg = plt.legend(["Error Radius Accuracy (\%)","Mean Sampled Error Radius (meters)"])
plt.xlabel("Algorithmic Error -- 95\% radius in Pixels")
plt.xticks(xaxis)

plt.tight_layout()

plt.savefig('figs/algorithm_influence.pdf',bbox_inches='tight')
plt.savefig('figs/algorithm_influence.pgf',bbox_inches='tight')


### 5. Examine How ground area ie. fov/resolution affects accuracy

In [28]:
# want to see how mean/median radius changes, as resolution increases
# test once with 90 degrees and once with 45 degrees

# 90 degrees

resolutions = np.array([100, 300, 500, 1000, 2000, 5000, 10000])
angles = np.array([90, 70, 50, 40, 32])
data = []

for angle in angles:
    data.append([])
    for res in resolutions:
        data[-1].append(do_eval(np.array([0.0, 0.0, 6.0]), 
            float(angle),
            0.0,
            alg_error=4.0,
            camera_res=(res,res),
            n_samples_per_camera=50,
            n_camera_instances=20,
            orientation_percent=0.90,
            pos_percent=0.9,
            alg_percent=0.9)
        )

In [31]:
plt.clf()
plt.figure(figsize=(6,3))
for j, angle in enumerate(angles[:-1]):
    angle_data = data[j]
    means, medians, stddevs, percentiles = [], [], [], []
    for i in range(len(resolutions)):
        res_data = angle_data[i]
        radii = res_data['predicted_error_radius'].ravel()
        mean, median, stddev, percentile = np.mean(radii), np.median(radii), np.var(radii)**2, np.percentile(radii, 99.0)
        means.append(mean)
        stddevs.append(stddev)
    
    # plot this curve
    plt.errorbar(resolutions, means, yerr=3*stddev)
    print("Angle: {0}, Mean radius across resolutions: {1}".format(angle, means))
    
leg = plt.legend([u"{}$^{{\circ}}$ pitch".format(a) for a in angles])
plt.xlabel("Horizontal and Vertical Resolution")
plt.ylabel("Mean Sampled Error Radius (meters)")
# plt.show()
plt.tight_layout()

plt.savefig('figs/resolution_angle_effects_mean.pdf',bbox_inches='tight')
plt.savefig('figs/resolution_angle_effects_mean.pgf',bbox_inches='tight')
plt.clf()

plt.figure(figsize=(6,3))
for j, angle in enumerate(angles[:-1]):
    angle_data = data[j]
    means, medians, stddevs, percentiles = [], [], [], []
    for i in range(len(resolutions)):
        res_data = angle_data[i]
        radii = res_data['predicted_error_radius'].ravel()
        mean, median, stddev, percentile = np.mean(radii), np.median(radii), np.var(radii)**2, np.percentile(radii, 99.0)
        medians.append(median)
        percentiles.append(percentile)
    
    # plot this curve
    plt.plot(resolutions, medians)
    print("Angle: {0}, Median radius across resolutions: {1}".format(angle, medians))

    
leg = plt.legend([u"{}$^{{\circ}}$ pitch".format(a) for a in angles])
plt.ylabel("Median Sampled Error Radius (meters)")
plt.xlabel("Horizontal and Vertical Resolution")
plt.xlim(0.0, 0.8)

plt.tight_layout()

plt.savefig('figs/resolution_angle_effects_median.pdf',bbox_inches='tight')
plt.savefig('figs/resolution_angle_effects_median.pgf',bbox_inches='tight')
plt.clf()

plt.figure(figsize=(6,3))
for j, angle in enumerate(angles[:-1]):
    angle_data = data[j]
    means, medians, stddevs, percentiles = [], [], [], []
    for i in range(len(resolutions)):
        res_data = angle_data[i]
        radii = res_data['predicted_error_radius'].ravel()
        mean, median, stddev, percentile = np.mean(radii), np.median(radii), np.var(radii)**2, np.percentile(radii, 95.0)
        percentiles.append(percentile)
    
    # plot this curve
    plt.plot(resolutions, percentiles)
    print("Angle: {0}, 95th percentile radius across resolutions: {1}".format(angle, percentiles))
    
leg = plt.legend([u"{}$^{{\circ}}$ pitch".format(a) for a in angles])
plt.xlabel("Horizontal and Vertical Resolution")
plt.ylabel("95th Percentile of Sampled Error Radius (meters)")
plt.xlim(0.0, 5.0)



plt.tight_layout()


plt.savefig('figs/resolution_angle_effects_99th_percentile.pdf',bbox_inches='tight')
plt.savefig('figs/resolution_angle_effects_99th_percentile.pgf',bbox_inches='tight')
plt.clf()


Angle: 90, Mean radius across resolutions: [0.4020133008487678, 0.19314730296447497, 0.15158797815491784, 0.11967141664113953, 0.10361595708892202, 0.094576869694491111, 0.090993179106301933]
Angle: 70, Mean radius across resolutions: [0.45120360027280093, 0.2175207776529004, 0.16949496058447125, 0.13590851575517299, 0.11640946264951132, 0.1044162413387387, 0.10364588370747951]
Angle: 50, Mean radius across resolutions: [0.72501820902364777, 0.33503689330052222, 0.27088888233985009, 0.21548723554127155, 0.18071530888494092, 0.16918516328321268, 0.16320831365950381]
Angle: 40, Mean radius across resolutions: [1.1919667922048676, 0.58107625110252681, 0.44507410835796229, 0.37276885441808055, 0.31194315915377036, 0.28600778044295477, 0.27402798016624724]
Angle: 90, Median radius across resolutions: [0.40170774600397363, 0.19354006770269738, 0.15195287762881809, 0.11974154915459981, 0.10346117670748645, 0.094759924256036049, 0.091078974739988744]
Angle: 70, Median radius across resolutions

In [30]:
resolutions

array([  100,   300,   500,  1000,  2000,  5000, 10000])